# **Tugas**

Prosedur pelatihan pada praktikum 2 merupakan prosedur sederhana, yang tidak memberi Anda banyak kendali. Model ini menggunakan "teacher-forcing" yang mencegah prediksi buruk diumpankan kembali ke model, sehingga model tidak pernah belajar untuk pulih dari kesalahan. Jadi, setelah Anda melihat cara menjalankan model secara manual, selanjutnya Anda akan mengimplementasikan custom loop pelatihan. Hal ini memberikan titik awal jika, misalnya, Anda ingin menerapkan pembelajaran kurikulum untuk membantu menstabilkan keluaran open-loop model. Bagian terpenting dari loop pelatihan khusus adalah fungsi langkah pelatihan.

Gunakan [tf.GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape) untuk men track nilai gradient. Anda dapat mempelajari lebih lanjut tentang pendekatan ini dengan membaca [eager execution guide.](https://www.tensorflow.org/guide/basics)

Prosedurnya adalah "
1. Jalankan Model dan hitung loss dengan [tf.GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape).
2. Hitung update dan terapkan pada model dengan optimize

# **Jawaban**

* Perbandingan dengan Praktikum 2

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [6]:
# Print unique characters
for char in vocab:
    print(char, end=' ')


   ! $ & ' , - . 3 : ; ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z a b c d e f g h i j k l m n o p q r s t u v w x y z 

In [7]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [8]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [9]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [10]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [11]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [12]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [13]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [14]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [15]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [16]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [17]:
seq_length = 100

In [18]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [19]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [20]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [21]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [22]:
dataset = sequences.map(split_input_target)

In [23]:
for input_example, target_example in dataset.take(1):
  print("Input :", text_from_ids(input_example).numpy())
  print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [24]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [25]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [26]:

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [27]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [29]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
sampled_indices = tf.random.categorical(example_batch_predictions[0],num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [31]:
sampled_indices

array([13,  9, 63,  9, 61,  9, 37, 56, 47,  0, 45, 17, 40,  3, 15, 64, 52,
       56, 44, 16, 51, 28, 42, 36, 30,  9, 54,  1,  8, 30, 21, 12, 16, 46,
       56, 41, 12, 49, 42, 28, 26, 12, 60, 54, 10, 41, 47, 27, 42, 19, 40,
       42, 24,  7, 49, 12, 24, 31, 20, 15,  0,  3, 35, 25, 42, 57, 62, 58,
       28,  6, 26, 45, 22, 22, 49, 56,  3, 46, 27, 28, 60, 43, 45, 18,  5,
       48, 10,  3, 48, 24, 31, 16, 32, 20,  4, 11, 10, 19,  2, 24])

In [32]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"g's own mouth, thereon\nHis execution sworn.\n\nPOLIXENES:\nI do believe thee:\nI saw his heart in 's fac"

Next Char Predictions:
 b"?.x.v.Xqh[UNK]fDa!BymqeClOcWQ.o\n-QH;Cgqb;jcOM;uo3bhNcFacK,j;KRGB[UNK]!VLcrwsO'MfIIjq!gNOudfE&i3!iKRCSG$:3F K"


In [33]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [34]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.190138, shape=(), dtype=float32)


In [35]:
tf.exp(example_batch_mean_loss).numpy()

66.03189

In [36]:
model.compile(optimizer='adam', loss=loss)

In [37]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [38]:
EPOCHS = 10

In [39]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 20s 55ms/step - loss: 2.7170
Epoch 2/10
172/172 [==============================] - 10s 51ms/step - loss: 1.9908
Epoch 3/10
172/172 [==============================] - 11s 51ms/step - loss: 1.7099
Epoch 4/10
172/172 [==============================] - 12s 52ms/step - loss: 1.5502
Epoch 5/10
172/172 [==============================] - 11s 53ms/step - loss: 1.4519
Epoch 6/10
172/172 [==============================] - 11s 53ms/step - loss: 1.3850
Epoch 7/10
172/172 [==============================] - 11s 55ms/step - loss: 1.3330
Epoch 8/10
172/172 [==============================] - 12s 54ms/step - loss: 1.2875
Epoch 9/10
172/172 [==============================] - 12s 55ms/step - loss: 1.2466
Epoch 10/10
172/172 [==============================] - 12s 55ms/step - loss: 1.2076


In [40]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [41]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [42]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Rought that you have you begin, 'O work? hawf you, nor Henry is thine;
And change with me now, then she all resolution
And bent true London in his son
his tomeness of no.

GREY:
The locker nature in this pity of a name
I prithee, rescure her her name of your own,
And with death shore. For himself women prince,
That by thy heep of Romeo, say therewere?

Bo'NON:
A dishes all unbonneting.

DUKE OF AUMERLE:
Meaning, my liene that is all of wincemary of Buckinghty
With wisdom in a mother of mony;
And so 'nglisber, behold that same, York;
While, whose half o'er runs and your counsello,
Who is perudua lay my death? and, in the air
And bedince would see more fellows in howor. Come on!
Let them not water with such burning,
Should not infected: at he looks through more
mot'st with us
And provers to expreselves when we have: But, freery is must
Which she hath your heart was love's preced?

NORTHUMBERLAND:
There is a hair bot! O Gremio, I was comploy
I sinner usbrest so find up.

TOLINGBAM:

In [43]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nOur suits, sirrah, go to the time I reib: with all\nthese travellers up his facter hearted with-hea?'st\no' the swellence old prosperose, that ourselves or you\nto presert ox our times Romeo'? thy shepherd.\n\nKING HENRY VI:\nWhy, ho?\nConstrain!\n\nLEONTES:\nOur earlshing shuts are beaution of hell,\nTheir mother put in mine\nOr in the subject villuces; teppet that womun,\nFrom faith and beg of gaze-morrial,\nKing Henry last of this? My father's head! most sorrow or restremits,\nWhich may hear make them things scolding perdence;\nFor never done spoke conspirach; but those us\nThanks, than he spake anong.\n\nKING LEWIS XI:\nAhis me not, whenefor Kelsies good strange, dream'd misach\nAnd pity them spowled the over-raports\nOf grues of Rome.'\nBy those should not prove a strange grow;\nWho hot because ungronsent these tender deserts,\nAnd peace them father and my master\nTo this place o' these self-hariture of bost?\nThou art those that which have a wash on two?\n\n

In [44]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [45]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
A vixos and true fence an aside comes
Than think not with the hignark of news, are more
Thousand ni


In [50]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
    inputs, labels = inputs
    with tf.GradientTape() as tape:
      predictions = self(inputs, training=True)
      loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

Kode diatas menerapkan train_step method sesuai dengan  [Keras' train_step conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). Ini opsional, tetapi memungkinkan Anda mengubah perilaku langkah pelatihan dan tetap menggunakan keras [Model.compile](https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile) and [Model.fits](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) methods.

In [51]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [52]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [53]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 15s 57ms/step - loss: 2.7164


In [54]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 11s 55ms/step - loss: 1.9808


Atau jika ingin lebih mengetahui dalamnya, kita bisa membuat custom training loop sendiri:

In [57]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
  start = time.time()

  mean.reset_states()
  for (batch_n, (inp, target)) in enumerate(dataset):
    logs = model.train_step([inp, target])
    mean.update_state(logs['loss'])

    if batch_n % 50 == 0:
      template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
      print(template)

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print()
  print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
  print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
  print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 1.8323
Epoch 1 Batch 50 Loss 1.7398
Epoch 1 Batch 100 Loss 1.6428
Epoch 1 Batch 150 Loss 1.6215

Epoch 1 Loss: 1.7043
Time taken for 1 epoch 20.47 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.6138
Epoch 2 Batch 50 Loss 1.5738
Epoch 2 Batch 100 Loss 1.5692
Epoch 2 Batch 150 Loss 1.4994

Epoch 2 Loss: 1.5454
Time taken for 1 epoch 13.31 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5263
Epoch 3 Batch 50 Loss 1.4357
Epoch 3 Batch 100 Loss 1.4820
Epoch 3 Batch 150 Loss 1.4541

Epoch 3 Loss: 1.4479
Time taken for 1 epoch 20.49 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.3863
Epoch 4 Batch 50 Loss 1.4196
Epoch 4 Batch 100 Loss 1.3597
Epoch 4 Batch 150 Loss 1.3539

Epoch 4 Loss: 1.3801
Time taken for 1 epoch 10.96 sec
_____________________________________________________________________

Jalankan kode diatas dan sebutkan perbedaanya dengan praktikum 2?

* Perbedaan diatas dengan Praktikum 2 ialah:

**prosedur pelatihan**

Pada praktikum 2, digunakan pendekatan pelatihan yang lebih sederhana dan umum dengan menggunakan fungsi model.fit. Sebaliknya, pada kode tugas, terdapat penerapan pendekatan pelatihan yang lebih spesifik dan kompleks. Pendekatan ini melibatkan definisi metode train_step dalam model turunan yang mengatur pelatihan pada tingkat batch. Di dalam train_step, dilakukan secara eksplisit perhitungan loss, perhitungan gradien, dan penerapan pembaruan bobot model dengan menggunakan apply_gradients. Selain itu, juga digunakan objek tf.metrics.Mean untuk menghitung rata-rata loss selama proses pelatihan.

Pendekatan yang diterapkan pada tugas memberikan lebih banyak kontrol dan fleksibilitas dalam mengatur proses pelatihan model. Hal ini memungkinkan untuk menyesuaikan langkah-langkah pelatihan sesuai kebutuhan spesifik, dan memungkinkan penggunaan metrik tambahan selain loss untuk pemantauan kinerja model.